## Revisit 1.3 Softmax


### Import TensorFlow and MNIST data 

In [6]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

### Read MNIST data and describe 

In [7]:
mnist = input_data.read_data_sets('/tmp/tensorflow/mnist/input_data', one_hot=True)
print(mnist)

Extracting /tmp/tensorflow/mnist/input_data\train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-labels-idx1-ubyte.gz
Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000001EFDD8F3D30>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000001EFDFEA6F98>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000001EFDFEA6F60>)


### Create linear model similar as linear regression without Softmax

In [8]:
x = tf.placeholder(tf.float32, [None, 784], name="image")
W = tf.Variable(tf.zeros([784, 10]), name="Weights")
b = tf.Variable(tf.zeros([10]), name="bias")
y = tf.matmul(x, W) + b

### Define the label 

In [9]:
labels = tf.placeholder(tf.float32, [None, 10], name="labels")

### Define cross entropy and optimizer

In [10]:
# Raw format should be
# tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)), reduction_indices=[1]))
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=y))
optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

### Train the model, notice we use mini batches instead of full batch training

In [12]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
# Export graph
writer = tf.summary.FileWriter("softmax_001", sess.graph)
for i in range(1000):
    if(i % 100 == 0):
        total_loss = 0
    batch_xs, batch_ys = mnist.train.next_batch(200) # Notice mini-batch training
    # Also note next_batch reshuffles the examples after going through all the them every epoch
    _, current_cross_entropy = sess.run([optimizer, cross_entropy], feed_dict={x: batch_xs, labels: batch_ys})
    total_loss += current_cross_entropy
    if(i % 100 == 1):
        print("loss for step %s: %s" %(i//100 + 1, total_loss))
writer.close()

loss for step 1: 4.59393954277
loss for step 2: 3.06299591064
loss for step 3: 2.39385461807
loss for step 4: 1.83114880323
loss for step 5: 1.75329369307
loss for step 6: 1.6099473834
loss for step 7: 1.4789814949
loss for step 8: 1.49051493406
loss for step 9: 1.28540229797
loss for step 10: 1.25435090065


### Test trained model and output performance

In [14]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, labels: mnist.test.labels}))

0.8699
